<a href="https://colab.research.google.com/github/kdwang1808/M5_LGB_WRMSSE/blob/master/M5_LGB_WRMSSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.21-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.21-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
import sys
os.chdir('drive/Colab Notebooks/M5_accuracy')

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sys
import gc
import time
import lightgbm as lgb
# import graphviz
from  datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# 减少内存占用空间
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
# create train data_sale 
train_start = 1
test_start = 1800
is_train = True
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float16" }
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
      "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int8",
      "month": "int8", "year": "int16", "snap_CA": "int8", 'snap_TX': 'int8', 'snap_WI': 'int8' }

start_day = train_start if is_train else test_start
numcols = [f"d_{day}" for day in range(start_day,1914)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
SALE_DTYPES = {numcol:"float32" for numcol in numcols} 
SALE_DTYPES.update({col: "category" for col in catcols if col != "id"})

price = pd.read_csv('sell_prices.csv',dtype=PRICE_DTYPES)
cal = pd.read_csv('calendar.csv',dtype=CAL_DTYPES)
sale = pd.read_csv('sales_train_validation.csv',dtype=SALE_DTYPES,usecols=catcols+numcols) #读取指定列
    
# 类别标签转换
for col, col_dtype in PRICE_DTYPES.items():
  if col_dtype == "category":
    price[col] = price[col].cat.codes.astype("int16")
    price[col] -= price[col].min()

cal["date"] = pd.to_datetime(cal["date"])
for col, col_dtype in CAL_DTYPES.items():
  if col_dtype == "category":
    cal[col] = cal[col].cat.codes.astype("int16")
    cal[col] -= cal[col].min()

for col in catcols:
  if col != "id":
    sale[col] = sale[col].cat.codes.astype("int16")
    sale[col] -= sale[col].min()

# 提交格式里有一部分为空
if not is_train:
  for day in range(1913+1, 1913+ 2*28 +1):
    sale[f"d_{day}"] = np.nan

sale = pd.melt(sale,
        id_vars = catcols,
        value_vars = [col for col in sale.columns if col.startswith("d_")],
        var_name = "d",
        value_name = "sales")
sale = reduce_mem_usage(sale)
sale = sale.merge(cal, on= "d", copy = False)
sale = sale.merge(price, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
sale.head(10)

In [0]:
def create_feature(sale, is_train=True, day=None):
    # 加入特征
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]

    # 如果是测试集只需要计算一天的特征，减少计算量
    # 注意训练集和测试集特征生成要一致
    if is_train:
        for lag, lag_col in zip(lags, lag_cols):
            sale[lag_col] = sale[["id","sales"]].groupby("id")["sales"].shift(lag)
    else:
        for lag, lag_col in zip(lags, lag_cols):
            sale.loc[sale.date == day, lag_col] = sale.loc[sale.date ==day-timedelta(days=lag), 'sales'].values  


    # 取7天前的数据，28天前的数据做移动平均
    wins = [7, 28]

    if is_train:
        for win in wins :
            for lag,lag_col in zip(lags, lag_cols):
                sale[f"rmean_{lag}_{win}"] = sale[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())
    else:
        for win in wins:
            for lag in lags:
                df_window = sale[(sale.date <= day-timedelta(days=lag)) & (sale.date > day-timedelta(days=lag+win))]
                df_window_grouped = df_window.groupby("id").agg({'sales':'mean'}).reindex(sale.loc[sale.date==day,'id'])
                sale.loc[sale.date == day,f"rmean_{lag}_{win}"] = df_window_grouped.sales.values   

    # 处理时间特征
    date_features = {"wday": "weekday",
            "week": "weekofyear",
            "month": "month",
            "quarter": "quarter",
            "year": "year",
            "mday": "day"}
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in sale.columns:
            sale[date_feat_name] = sale[date_feat_name].astype("int16")
        else:
            sale[date_feat_name] = getattr(sale["date"].dt, date_feat_func).astype("int16")
    return sale

In [0]:
# create feature_sale
is_train=True
day=None
lags = [7, 28]
lag_cols = [f"lag_{lag}" for lag in lags ]

# 如果是测试集只需要计算一天的特征，减少计算量
# 注意训练集和测试集特征生成要一致
if is_train:
  for lag, lag_col in zip(lags, lag_cols):
    sale[lag_col] = sale[["id","sales"]].groupby("id")["sales"].shift(lag)
else:
  for lag, lag_col in zip(lags, lag_cols):
    sale.loc[sale.date == day, lag_col] = sale.loc[sale.date ==day-timedelta(days=lag), 'sales'].values  

# 将获取7天前的数据，28天前的数据做移动平均
wins = [7, 28]
if is_train:
  for win in wins :
    for lag,lag_col in zip(lags, lag_cols):
      sale[f"rmean_{lag}_{win}"] = sale[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())
else:
  for win in wins:
    for lag in lags:
      df_window = sale[(sale.date <= day-timedelta(days=lag)) & (sale.date > day-timedelta(days=lag+win))]
      df_window_grouped = df_window.groupby("id").agg({'sales':'mean'}).reindex(sale.loc[sale.date==day,'id'])
      sale.loc[sale.date == day,f"rmean_{lag}_{win}"] = df_window_grouped.sales.values   

# 处理时间特征
# 有的时间特征没有，通过datetime的方法自动生成
date_features = {"wday": "weekday",
         "week": "weekofyear",
         "month": "month",
         "quarter": "quarter",
         "year": "year",
         "mday": "day"}
for date_feat_name, date_feat_func in date_features.items():
  if date_feat_name in sale.columns:
    sale[date_feat_name] = sale[date_feat_name].astype("int16")
  else:
    sale[date_feat_name] = getattr(sale["date"].dt, date_feat_func).astype("int16")
sale.head(10)

In [0]:
# https://www.kaggle.com/chrisrichardmiles/m5-flexible-custom-metric-lv-12-only-for-lgbm
# 根据Level_12自定义损失函数(待修改)
def L12_WRMSSE(preds, actuals, p_horizon, num_products, scale, weight): 
    
    actuals = actuals[-(p_horizon * num_products):]
    preds = preds[-(p_horizon * num_products):]
    diff = actuals - preds
    res = diff ** 2
    res = res/scale.values
    res = res
    res = res.reshape(p_horizon, num_products)
    res = res.mean(axis=0)
    res = np.sqrt(res)
    res = res * weight
    res = res.sum()
    return res

def get_weights_scales_level_12(df, end_test, path):
  
    wdf = pd.read_csv(f'{path}weight_scale_{end_test-27}.csv')
    wdf['scaled_weight'] = wdf.weight/np.sqrt(wdf.scale)
    wdf = wdf[wdf.Level_id == 'Level12']
    wdf['id'] = wdf['Agg_Level_1'] + '_' +  wdf['Agg_Level_2'] + '_validation'
    wdf = wdf[['id', 'scale', 'weight', 'scaled_weight']]
    wdf = pd.merge(df[['id']], wdf, on='id', how='left')
    
    return wdf

P_HORIZON = 28         # Prediction horizon 
NUM_PRODUCTS = 30490      # Number of products 

END_TEST = 1913
train_mask = sale['d'].apply(lambda x:x[2:])<=(END_TEST-28)
test_mask = sale['d'].apply(lambda x:x[2:])>(END_TEST-28)
train_valid_mask = train_mask & (sale['d'].apply(lambda x:x[2:])>(END_TEST-56))

wdf = get_weights_scales_level_12(sale, END_TEST, path)
scale = wdf[train_valid_mask].scale
weight = wdf[train_valid_mask].weight[:NUM_PRODUCTS]

################### Custom metric #####################
def custom_metric(preds, train_data):
    actuals = train_data.get_label()
    res = L12_WRMSSE(preds, actuals, P_HORIZON, NUM_PRODUCTS, scale, weight)
    return 'L12_WRMSSE', res, False

In [0]:
# 清洗数据
sale.dropna(inplace=True)
cat_feats = ['item_id', 'dept_id', 'store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]+["price_digit"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = sale.columns[~sale.columns.isin(useless_cols)]
X_train = sale[train_cols]
y_train = sale["sales"]
X_train.head(10)

In [0]:
train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
valid_data = lgb.Dataset(X_train.iloc[valid_inds], label = y_train.iloc[valid_inds],categorical_feature=cat_feats, free_raw_data=False)
valid_inds = np.random.choice(len(X_train), 10000)

In [0]:
# 训练lgb
t0 = time.time()
params = {
      "objective" : "tweedie",
      "tweedie_variance_power" : 1.18,
      "metric" :"rmse",
      "boosting_type": 'gbdt',
      "force_row_wise" : True,
      "learning_rate" : 0.075,
      "sub_feature" : 0.8,
      "sub_row" : 0.75,
      "bagging_freq" : 1,
      "lambda_l2" : 0.1,
      "nthread": -1,
      "verbosity": 1,
      "num_iterations" : 5000,
      "early_stopping_round" : 300,
      "num_leaves": 127,
      "min_data_in_leaf": 104,
      }
evals_result = {}

# m_lgb = lgb.train(params, train_data, valid_sets = [valid_data], evals_result=evals_result, feval=custom_metric, verbose_eval=25)
m_lgb = lgb.train(params, train_data, valid_sets = [valid_data], evals_result=evals_result, verbose_eval=25)
t1 = time.time()
print("Training time: ")
print(t1-t0)

In [0]:
# create test data_test_data 
train_start = 1
test_start = 1800
is_train = False
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float16" }
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
      "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int8",
      "month": "int8", "year": "int16", "snap_CA": "int8", 'snap_TX': 'int8', 'snap_WI': 'int8' }

start_day = train_start if is_train else test_start
numcols = [f"d_{day}" for day in range(start_day,1914)]
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
SALE_DTYPES = {numcol:"float32" for numcol in numcols} 
SALE_DTYPES.update({col: "category" for col in catcols if col != "id"})

price = pd.read_csv('sell_prices.csv',dtype=PRICE_DTYPES)
cal = pd.read_csv('calendar.csv',dtype=CAL_DTYPES)
test_data = pd.read_csv('sales_train_validation.csv',dtype=SALE_DTYPES,usecols=catcols+numcols) #usecols读取指定列
    
# 类别标签转换
for col, col_dtype in PRICE_DTYPES.items():
  if col_dtype == "category":
    price[col] = price[col].cat.codes.astype("int16")
    price[col] -= price[col].min()

cal["date"] = pd.to_datetime(cal["date"])
for col, col_dtype in CAL_DTYPES.items():
  if col_dtype == "category":
    cal[col] = cal[col].cat.codes.astype("int16")
    cal[col] -= cal[col].min()

for col in catcols:
  if col != "id":
    test_data[col] = test_data[col].cat.codes.astype("int16")
    test_data[col] -= test_data[col].min()

# 提交格式里有一部分为空
if not is_train:
  for day in range(1913+1, 1913+ 2*28 +1):
    test_data[f"d_{day}"] = np.nan

test_data = pd.melt(test_data,
        id_vars = catcols,
        value_vars = [col for col in test_data.columns if col.startswith("d_")],
        var_name = "d",
        value_name = "sales")
test_data = reduce_mem_usage(test_data)
test_data = test_data.merge(cal, on= "d", copy = False)
test_data = test_data.merge(price, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [0]:
# 加入趋势alpha
date = datetime(2016, 4, 25) 
alpha = [1.035]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):
  test_data_c = test_data.copy()
  cols = [f"F{i}" for i in range(1,29)]
  for i in range(0, 28):
    day = date + timedelta(days=i)
    print(i, day)
    tst = test_data_c[(test_data_c.date >= day - timedelta(days=57)) & (test_data_c.date <= day)].copy()
    tst = create_feature(tst, is_train=False, day=day)
    tst = tst.loc[tst.date == day , train_cols]

    test_data_c.loc[test_data_c.date == day, "sales"] = alpha*m_lgb.predict(tst)

  # 提交数据的格式
  test_sub = test_data_c.loc[test_data_c.date >= date, ["id", "sales"]].copy()
  test_sub["F"] = [f"F{rank}" for rank in test_sub.groupby("id")["id"].cumcount()+1]
  test_sub = test_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
  test_sub.fillna(0., inplace = True)
  test_sub.sort_values("id", inplace = True)
  test_sub.reset_index(drop=True, inplace = True)
  if icount == 0 :
    sub = test_sub
    sub[cols] *= weight
  else:
    sub[cols] += test_sub[cols]*weight
  print(icount, alpha, weight)
    
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.head()
# sub.to_csv("LGB_v1.0.csv",index=False)